In [144]:
import keras.losses
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
from keras.preprocessing.image import img_to_array
from tensorflow.keras import layers
from keras.regularizers import l2
import numpy as np
from keras.layers import Dense, BatchNormalization, Dropout
from keras.models import Sequential
import sklearn as skl
from sklearn.decomposition import TruncatedSVD, PCA
import datetime

In [145]:
model = VGG16(include_top=True)

In [146]:
def get_features(ID):
    img_path = 'food_porn/food/' + ID + '.jpg'
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    features = np.array(model.predict(x))
    return features

In [147]:
%%time
data = np.zeros((10000,1001))
id = np.array(range(0,10000))
data[:,0] = id.transpose()

def features():
    for i in range(0,10000):
        featureid = '{0:05}'.format(i)
        features = get_features(featureid)
        data[i,1:] = features
        if i % 100 == True:
            print(i)
    return data

#data = features()

Wall time: 21.3 ms


In [148]:
#save_ass = 'features_vgg16.csv'

In [149]:
#print(np.count_nonzero(data))
#save_ass = 'features_vgg16.csv'

#np.savetxt(save_ass, data, delimiter=',')

using false we got:     527 correct
using true we got:      520 correct

how about... we use the true version to feed a NN

In [150]:
train_trip = np.loadtxt("train_triplets.txt").astype('int')

data = np.genfromtxt('features_vgg16.csv', delimiter= ',')
data = np.array(data[:,1:])

reduction = 256

temp_data = np.round(data,decimals=4)
temp =0
for i in range(0,1000):
    if np.count_nonzero(temp_data[:,i])==0:
        print("Empty Column:", i)
        temp=temp+1
print(temp)

for i in range(10000):
    if np.count_nonzero(temp_data[i,:])==0:
        print("Empty Row:", i)

svd = TruncatedSVD(n_components=reduction, algorithm='arpack')
svd.fit(data)
transformed = svd.transform(data)
print(transformed.shape)

def load_features(int_id):
    return transformed[int_id]

In [151]:
pca = PCA(n_components=reduction, svd_solver='arpack')
pca.fit(data)
transformed = pca.transform(data)
print(transformed.shape)

def load_features(int_id):
    return transformed[int_id]

(10000, 512)


In [152]:
%%time

#create the base dataset with the 2 sets of labels
data_y1 = np.ones((59515,1))
data_y0 = np.zeros((59515,1))

#create the inverse dataset:
train_trip_inv = np.zeros((59515,3)).astype('int')
train_trip_inv[:,0] = train_trip[:,0]
train_trip_inv[:,1] = train_trip[:,2]
train_trip_inv[:,2] = train_trip[:,1]

#assemble everything into one huge pile of S*** in order to shuffle it
id_fin = np.concatenate((train_trip,train_trip_inv),axis=0)
labels = np.concatenate((data_y1,data_y0),axis=0)
id_labels = np.concatenate((id_fin,labels),axis=1)

data_nn_x = np.zeros((119030,3*reduction))

for i in range(0,3):
        for j in range(0,119030):
            data_nn_x[j,range(i*reduction,reduction*(i+1))] = load_features(id_labels[j,i].astype('int'))
            if j%10000==False:
                print("we're at entry:", i, j)


#print(train_trip)
#print(train_trip_inv)
#now we try to randomize the data in order to increase our accuracy
# we may even double the dataset by using 0 as well as the 1 case

we're at entry: 0 0
we're at entry: 0 10000
we're at entry: 0 20000
we're at entry: 0 30000
we're at entry: 0 40000
we're at entry: 0 50000
we're at entry: 0 60000
we're at entry: 0 70000
we're at entry: 0 80000
we're at entry: 0 90000
we're at entry: 0 100000
we're at entry: 0 110000
we're at entry: 1 0
we're at entry: 1 10000
we're at entry: 1 20000
we're at entry: 1 30000
we're at entry: 1 40000
we're at entry: 1 50000
we're at entry: 1 60000
we're at entry: 1 70000
we're at entry: 1 80000
we're at entry: 1 90000
we're at entry: 1 100000
we're at entry: 1 110000
we're at entry: 2 0
we're at entry: 2 10000
we're at entry: 2 20000
we're at entry: 2 30000
we're at entry: 2 40000
we're at entry: 2 50000
we're at entry: 2 60000
we're at entry: 2 70000
we're at entry: 2 80000
we're at entry: 2 90000
we're at entry: 2 100000
we're at entry: 2 110000
Wall time: 18 s


In [153]:
collab = np.concatenate((data_nn_x,labels), axis=1)
np.random.shuffle(collab)

data_x = collab[:,:3*reduction]
data_y = collab[:,3*reduction]

In [154]:
#print("%d bytes" % (data_nn_x.size * data_nn_x.itemsize))
#np.savetxt("nn_data.csv",data_nn_x,delimiter=',')

In [155]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


layer = layers.Dense(
    units=64,
    kernel_regularizer=regularizers.l1_l2(l1=1e-2, l2=1e-2),
    bias_regularizer=regularizers.l2(1e-2),
    activity_regularizer=regularizers.l2(1e-2)
)

In [156]:
#log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
#tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

#normalize the data:
print(data_x.shape)
print(np.min(data_x))
print(np.max(data_x))
print(np.mean(data_x))
print(np.count_nonzero(data_x<0))
print(np.count_nonzero(data_x))

temp_data = np.round(data_x,decimals=1)
data_x = np.round(data_x,decimals=1)

(119030, 1536)
-0.6010278384028279
0.9335400281491291
5.918253423436879e-06
92005814
182830080


normalise data in order to improve initial layer

perform dimensionality reduction on input set - DONE

test out different optimizers like SGD, adam etc.

there are no empty columns in the data (decimals=4) - DONE

In [157]:
%%time

#TAKE IT Branda:
neuralNetwork = Sequential()
neuralNetwork.add(Dense(256, activation='relu', input_dim=3*reduction, kernel_regularizer=l2(0.003)))
neuralNetwork.add(BatchNormalization())
neuralNetwork.add(Dropout(0.5))
neuralNetwork.add(Dense(256, activation='relu', kernel_regularizer=l2(0.003)))
neuralNetwork.add(BatchNormalization())
neuralNetwork.add(Dropout(0.4))
neuralNetwork.add(Dense(256, activation='relu', kernel_regularizer=l2(0.003)))
neuralNetwork.add(BatchNormalization())
neuralNetwork.add(Dropout(0.4))
neuralNetwork.add(Dense(256, activation='relu', kernel_regularizer=l2(0.003)))
neuralNetwork.add(BatchNormalization())
neuralNetwork.add(Dropout(0.3))
neuralNetwork.add(Dense(128, activation='relu', kernel_regularizer=l2(0.003)))
neuralNetwork.add(BatchNormalization())
neuralNetwork.add(Dropout(0.3))
neuralNetwork.add(Dense(64, activation='relu', kernel_regularizer=l2(0.003)))
neuralNetwork.add(BatchNormalization())
neuralNetwork.add(Dropout(0.2))
neuralNetwork.add(Dense(32, activation='relu', kernel_regularizer=l2(0.003)))
neuralNetwork.add(BatchNormalization())
neuralNetwork.add(Dense(16, activation='relu', kernel_regularizer=l2(0.003)))
neuralNetwork.add(BatchNormalization())
neuralNetwork.add(Dense(1, activation='sigmoid'))
#neuralNetwork.add(BatchNormalization())

neuralNetwork.compile(loss = keras.losses.BinaryCrossentropy(), optimizer='adam', metrics=['accuracy'])

#fit the network to (for now the un-sparse matrix)
neuralNetwork.fit(data_x, data_y, epochs=128, batch_size=1024, verbose=2, validation_split = 0.2)

neuralNetwork.summary()

Epoch 1/128
93/93 - 1s - loss: 4.0708 - accuracy: 0.5022 - val_loss: 3.1744 - val_accuracy: 0.4975
Epoch 2/128
93/93 - 1s - loss: 2.5885 - accuracy: 0.5033 - val_loss: 2.0640 - val_accuracy: 0.4975
Epoch 3/128
93/93 - 1s - loss: 1.7044 - accuracy: 0.5088 - val_loss: 1.4012 - val_accuracy: 0.4975
Epoch 4/128
93/93 - 1s - loss: 1.1990 - accuracy: 0.5292 - val_loss: 1.0454 - val_accuracy: 0.4975
Epoch 5/128
93/93 - 1s - loss: 0.9268 - accuracy: 0.5800 - val_loss: 0.8866 - val_accuracy: 0.4975
Epoch 6/128
93/93 - 1s - loss: 0.7867 - accuracy: 0.6289 - val_loss: 0.7914 - val_accuracy: 0.5200
Epoch 7/128
93/93 - 1s - loss: 0.7244 - accuracy: 0.6437 - val_loss: 0.7601 - val_accuracy: 0.5379
Epoch 8/128
93/93 - 1s - loss: 0.6982 - accuracy: 0.6521 - val_loss: 0.7147 - val_accuracy: 0.6420
Epoch 9/128
93/93 - 1s - loss: 0.6886 - accuracy: 0.6557 - val_loss: 0.7023 - val_accuracy: 0.6501
Epoch 10/128
93/93 - 1s - loss: 0.6846 - accuracy: 0.6634 - val_loss: 0.6899 - val_accuracy: 0.6596
Epoch 11/

In [158]:
%tensorboard --logdir logs/fit

Launching TensorBoard...

In [159]:
test_trip = np.loadtxt("test_triplets.txt").astype('int')
test_nn_x = np.zeros((59544,3*reduction))
for i in range(0,3):
        for j in range(0,59544):
            test_nn_x[j,range(i*reduction,reduction*(i+1))] = load_features(test_trip[j,i].astype('int'))
            if j%10000==False:
                print("we're at entry:", i, j)

predict_test_nn = neuralNetwork.predict(test_nn_x)

we're at entry: 0 0
we're at entry: 0 10000
we're at entry: 0 20000
we're at entry: 0 30000
we're at entry: 0 40000
we're at entry: 0 50000
we're at entry: 1 0
we're at entry: 1 10000
we're at entry: 1 20000
we're at entry: 1 30000
we're at entry: 1 40000
we're at entry: 1 50000
we're at entry: 2 0
we're at entry: 2 10000
we're at entry: 2 20000
we're at entry: 2 30000
we're at entry: 2 40000
we're at entry: 2 50000


In [160]:
export = np.array(predict_test_nn>0.5).astype('int')
np.savetxt("submission_vgg16_PCA_v1.txt",np.round(export,decimals=0), fmt='%i')